# LLM Exploration

In [3]:
import vertexai

vertexai.init(project="822479462122", location="us-central1")

## Simple conversation

In [7]:
from langchain.llms import VertexAI
from langchain.memory import ConversationTokenBufferMemory
from langchain.chains import ConversationChain

llm = VertexAI()
chain = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=60),
    verbose=True,
)

In [3]:
chain.run("What do you think of Google Cloud?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What do you think of Google Cloud?
AI:

> Finished chain.


'Google Cloud is a suite of cloud computing services that runs on the same infrastructure that Google uses for its end-user products, such as Google Search and YouTube. It offers a broad range of services, including computing, data storage, data analytics, and machine learning.'

## Conversation Image caption

In [4]:
from langchain.document_loaders import ImageCaptionLoader

loader = ImageCaptionLoader(
    path_images=[
        "https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Thure_de_Thulstrup_-_Battle_of_Shiloh.jpg/251px-Thure_de_Thulstrup_-_Battle_of_Shiloh.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Passion_fruits_-_whole_and_halved.jpg/270px-Passion_fruits_-_whole_and_halved.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Messier83_-_Heic1403a.jpg/277px-Messier83_-_Heic1403a.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/2022-01-22_Men%27s_World_Cup_at_2021-22_St._Moritz%E2%80%93Celerina_Luge_World_Cup_and_European_Championships_by_Sandro_Halank%E2%80%93257.jpg/288px-2022-01-22_Men%27s_World_Cup_at_2021-22_St._Moritz%E2%80%93Celerina_Luge_World_Cup_and_European_Championships_by_Sandro_Halank%E2%80%93257.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Wiesen_Pippau_%28Crepis_biennis%29-20220624-RM-123950.jpg/224px-Wiesen_Pippau_%28Crepis_biennis%29-20220624-RM-123950.jpg",
    ]
)
list_docs = loader.load()
list_docs

/workspaces/mywaifu/.direnv/python-3.10.11/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[Document(page_content='an image of a painting of a battle scene [SEP]', metadata={'image_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Thure_de_Thulstrup_-_Battle_of_Shiloh.jpg/251px-Thure_de_Thulstrup_-_Battle_of_Shiloh.jpg'}),
 Document(page_content='an image of a passion fruit and a half cut passion [SEP]', metadata={'image_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Passion_fruits_-_whole_and_halved.jpg/270px-Passion_fruits_-_whole_and_halved.jpg'}),
 Document(page_content='an image of the spiral galaxy [SEP]', metadata={'image_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Messier83_-_Heic1403a.jpg/277px-Messier83_-_Heic1403a.jpg'}),
 Document(page_content='an image of a man on skis in the snow [SEP]', metadata={'image_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/2022-01-22_Men%27s_World_Cup_at_2021-22_St._Moritz%E2%80%93Celerina_Luge_World_Cup_and_European_Championships_by_Sandro_Halank%E2%80%93257

## Load loader in an index

In [21]:
from langchain.vectorstores import Chroma
from langchain.embeddings import VertexAIEmbeddings

embedding = VertexAIEmbeddings()
vectorstore = Chroma.from_documents(list_docs, embedding)

In [40]:
from langchain.llms import VertexAI
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    return_source_documents=True,
    verbose=True,
)

In [41]:
qa({"question": "Show me an image that contain snow ?", "chat_history": []})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

an image of a man on skis in the snow [SEP]

an image of the spiral galaxy [SEP]

an image of a flower in the dark [SEP]

an image of a painting of a battle scene [SEP]

Question: Show me an image that contain snow ?
Helpful Answer:

> Finished chain.

> Finished chain.


{'question': 'Show me an image that contain snow ?',
 'chat_history': [],
 'answer': 'an image of a man on skis in the snow',
 'source_documents': [Document(page_content='an image of a man on skis in the snow [SEP]', metadata={'image_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/2022-01-22_Men%27s_World_Cup_at_2021-22_St._Moritz%E2%80%93Celerina_Luge_World_Cup_and_European_Championships_by_Sandro_Halank%E2%80%93257.jpg/288px-2022-01-22_Men%27s_World_Cup_at_2021-22_St._Moritz%E2%80%93Celerina_Luge_World_Cup_and_European_Championships_by_Sandro_Halank%E2%80%93257.jpg'}),
  Document(page_content='an image of the spiral galaxy [SEP]', metadata={'image_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Messier83_-_Heic1403a.jpg/277px-Messier83_-_Heic1403a.jpg'}),
  Document(page_content='an image of a flower in the dark [SEP]', metadata={'image_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Wiesen_Pippau_%28Crepis_biennis%29-20220624-RM-

## Load documents from Google Drive

In [1]:
from langchain.document_loaders import GoogleDriveLoader

loader = GoogleDriveLoader(
    folder_id="1daABjn2QXHMFUK_LUvVRlbUdTTc8nOWe",
    recursive=True,
)

In [2]:
list_docs = loader.load()
list_docs

[Document(page_content='www.it-ebooks.info', metadata={'source': 'https://drive.google.com/file/d/1zvuQfomh-OKq_kkUc_BiLZPhuvLoT2l5/view', 'title': 'Programming Erlang, 2nd Edition Software for a Concurrent World .pdf', 'page': 0}),
 Document(page_content='www.it-ebooks.info', metadata={'source': 'https://drive.google.com/file/d/1zvuQfomh-OKq_kkUc_BiLZPhuvLoT2l5/view', 'title': 'Programming Erlang, 2nd Edition Software for a Concurrent World .pdf', 'page': 1}),
 Document(page_content='Early Praise for\nProgramming Erlang, Second Edition\nThis second edition of Joe ’s seminal Programming Erlang  is a welcome update,\ncovering not only the core language and framework fundamentals but also key\ncommunity projects such as rebar and cowboy. Even experienced Erlang program -\nmers will find helpful tips and new insights throughout the book, and beginners\nto the language will appreciate the clear and methodical way Joe introduces and\nexplains key language concepts.\n➤Alexander Gounares\nFor

## Load all Google Drive directory to Vector Store

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import VertexAIEmbeddings

embedding = VertexAIEmbeddings()
vectorstore = Chroma.from_documents(list_docs[:10], embedding)

In [8]:
from langchain.llms import VertexAI
from langchain.chains import ConversationalRetrievalChain

llm = VertexAI()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    return_source_documents=True,
    verbose=True,
)

In [9]:
qa({"question": "Write an supervision tree in Erlang", "chat_history": []})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Early Praise for
Programming Erlang, Second Edition
This second edition of Joe ’s seminal Programming Erlang  is a welcome update,
covering not only the core language and framework fundamentals but also key
community projects such as rebar and cowboy. Even experienced Erlang program -
mers will find helpful tips and new insights throughout the book, and beginners
to the language will appreciate the clear and methodical way Joe introduces and
explains key language concepts.
➤Alexander Gounares
Former AOL CTO, advisor to Bill Gates, and founder/CEO of Concurix Corp.
A gem; a sensible, practical introduction to functional programming.
➤Gilad Bracha
Coauthor of the Java language and Java Virtual Machine specifications, creat

{'question': 'Write an supervision tree in Erlang',
 'chat_history': [],
 'answer': '```\n-module(my_app).\n-behaviour(supervisor).\n\n-export([start_link/0]).\n\nstart_link() ->\n    supervisor:start_link({local, ?MODULE}, ?MODULE, []).\n\n%% Behavioural functions\n\ninit([]) ->\n    SupFlags = {one_for_one, 5, 5},\n    Children = [\n        {my_worker, {my_worker, start_link, []},\n         permanent, 5000, supervisor, [my_worker]},\n',
 'source_documents': [Document(page_content='Early Praise for\nProgramming Erlang, Second Edition\nThis second edition of Joe ’s seminal Programming Erlang  is a welcome update,\ncovering not only the core language and framework fundamentals but also key\ncommunity projects such as rebar and cowboy. Even experienced Erlang program -\nmers will find helpful tips and new insights throughout the book, and beginners\nto the language will appreciate the clear and methodical way Joe introduces and\nexplains key language concepts.\n➤Alexander Gounares\nForme